In [274]:
from zipfile import ZipFile
from pathlib import Path

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

In [275]:
# import cupy 
# import cudf

In [276]:
df_dev = pd.read_csv(Path.cwd()/'data'/'DSL2122_january_dataset'/'development.csv')
df_eval = pd.read_csv(Path.cwd()/'data'/'DSL2122_january_dataset'/'evaluation.csv')

In [277]:
df_eval.shape

(74999, 5)

In [278]:
df = df_dev.append(df_eval)

In [279]:
# df['date'] = df['date'].\
#     astype('string')\
#     .str.split(' ')\
#     .apply(lambda x : ' '.join([x[i] for i in [1,2,3,5]]))\
#     .pipe(pd.to_datetime)

### Stopwords analysis: sklearn, nltk, stop_words

#### nltk

In [280]:
from nltk import download as nltk_download

nltk_download('stopwords')

from nltk.corpus import stopwords

nltk_stopwords = stopwords.words('english')

[nltk_data] Downloading package stopwords to /home/edoch/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#### sklearn

In [281]:
from sklearn.feature_extraction import text

sklearn_stopwords = list(text.ENGLISH_STOP_WORDS)

## Preprocessing

Tokenization

In [282]:
df['text_token'] = df['text'].str.split()

Removal of '@words', '&words', 'http:words'

In [283]:
df['text_noAt'] = df['text_token'].apply(lambda x : [i for i in x if not(i.startswith('@'))])\
    .apply(lambda x : [i for i in x if not(i.startswith('&'))])\
    .apply(lambda x : [i for i in x if not(i.startswith('http:'))])

Removal of '&amp', '&quot'

In [284]:
df['text_noAmpQuot'] = df['text_noAt']\
    .apply(lambda x : [i for i in x if '&amp' not in i])\
    .apply(lambda x : [i for i in x if '&quot' not in i])

Removal of repeated letters

In [285]:
a = ['ulaaa','bubu']
import re
[re.sub(r'(.)\1+', r'\1', word) for word in ['ulaaa','bubu']]

['ula', 'bubu']

In [286]:
import re 
df['text_duplLetters'] = df['text_noAmpQuot']\
    .apply(lambda x : [re.sub(r'(.)\1+', r'\1', word) for word in x])

Remove puntuation

In [287]:
import string as py_string
# df['text_noPunct'] = df['text_noAmpQuot']\
df['text_noPunct'] = df['text_duplLetters']\
    .apply(lambda x : [i.translate(str.maketrans('', '', py_string.punctuation)) for i in x])\
    .apply(lambda x : [i for i in x if i != ''])

Stemming and lemmatization

In [288]:
from nltk.stem import PorterStemmer, LancasterStemmer, SnowballStemmer, WordNetLemmatizer

# nltk_download('wordnet')
# nltk_download('omw-1.4')
# nltk_lemmatizer = WordNetLemmatizer()

nltk_stemmer = PorterStemmer()
# nltk_stemmer = LancasterStemmer()
# nltk_stemmer = SnowballStemmer('english') 

df['text_stemmed'] = df['text_noPunct'].apply(lambda x : [nltk_stemmer.stem(word) for word in x])
# df['text_stemmed'] = df['text_noPunct'].apply(lambda x : [nltk_lemmatizer.lemmatize(word) for word in x])

Negation

In [289]:
from nltk.sentiment.util import mark_negation

df['text_neg'] = df['text_noPunct'].apply(lambda x : mark_negation(x))
df['text_stemmed_neg'] = df['text_stemmed'].apply(lambda x : mark_negation(x))

Removal of stopwords

In [290]:
from stop_words import get_stop_words

stop_words_stopwords = get_stop_words('english')

considered_stopwords = stop_words_stopwords + [f'{word}_neg' for word in stop_words_stopwords]

df['text_noStopwords'] = df['text_neg'].apply(lambda x : [i for i in x if i not in considered_stopwords])

User manual filter

In [291]:
df = df.loc[
    (~(df['user'] == 'lost_dog') &
    ~(df['user'] == 'webwoke') &
    ~(df['user'] == 'tweetpet') &
    ~(df['user'].str.contains('tweeteradder')) &
    ~(df['user'].str.contains('tweetfollow')) &
    ~(df['user'] == 'divxdownloads')) |
    df['sentiment'].isna()
]

In [292]:
df[df['sentiment'].isna()].shape

(74999, 15)

## Model creation

In [293]:
from sklearn.model_selection import ParameterGrid

linearSVC_params = {
    'penalty' : ['l1'], #['l1','l2']
    'dual' : [False],
    'tol' : [1e-3,1e-4],
    'fit_intercept' : [False],
    'class_weight' : ['balanced'],
    'max_iter' : [100,300],
    'random_state' : [42],
    'C' : [1,10,50]
}

TfidfVectorizer_params = {
    'stop_words' : [None],
    'ngram_range' : [(1,1)],
    'max_features' : [None, 15000],
    'max_df' : [1.0,0.1,0.005],
    'min_df' : [1,0.00001],
    'binary' : [True,False],
    'norm' : ['l1','l2'],
    'use_idf' : [True],
    'smooth_idf' : [True,False],
    'sublinear_tf' : [False]
}

In [294]:
print(len(list(ParameterGrid(TfidfVectorizer_params)))*len(list(ParameterGrid(linearSVC_params))))

print(len(list(ParameterGrid(TfidfVectorizer_params)))*len(list(ParameterGrid(linearSVC_params)))*30/60/60)

1152
9.6


# Grid search

In [295]:
# from sklearn.model_selection import GridSearchCV
# from sklearn.pipeline import Pipeline
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.svm import LinearSVC
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import f1_score, classification_report, confusion_matrix

# df_final = df
# # df_final['text_final'] = df_final['text_noPunct'].apply(lambda x : ' '.join(x))
# df_final['text_final'] = df_final['text_neg'].apply(lambda x : ' '.join(x))

# mask_train_test = df_final['sentiment'].notna()

# X_train_valid = df_final.loc[mask_train_test,:]['text_final'].values
# y_train_valid = df_final.loc[mask_train_test,:]['sentiment'].values
# X_test = df_final.loc[~mask_train_test,:]['text_final'].values

# X_train, X_valid, y_train, y_valid = train_test_split(
#     X_train_valid, 
#     y_train_valid, 
#     shuffle=True, 
#     # stratify=y_train_valid, 
#     train_size=0.9, 
#     random_state=42
# )

# vectorizer = 'tfidf'
# model = 'linearSVC'

# pipe = Pipeline([
#     (vectorizer, TfidfVectorizer()),
#     (model, LinearSVC())
# ])

# def params_for_GridSearchCV(params_IN,step_name):
#     return {f'{step_name}__{key}':value for (key,value) in params_IN.items()}

# params_all = params_for_GridSearchCV(TfidfVectorizer_params,vectorizer)
# params_all.update(params_for_GridSearchCV(linearSVC_params,model))
# print(params_all)

# gscv = GridSearchCV(pipe, params_all, cv = 3, verbose = 4, scoring='f1_macro')

# gscv.fit(X_train, y_train)

# f1 = f1_score(y_valid, gscv.predict(X_valid),average='macro')
# report = classification_report(y_valid, gscv.predict(X_valid))
# confusion = confusion_matrix(y_valid, gscv.predict(X_valid))

# print(f1)
# print(report)
# print(confusion)

# from pathlib import Path
# from datetime import datetime

# results_name = Path.cwd()/'gscv_results'/(str(datetime.now()).replace('.','-').replace(':','-')+'.csv')

# pd.DataFrame(gscv.cv_results_).to_csv(results_name)

In [296]:
# a = pd.read_csv('gscv_results\\2022-01-12 20-43-02-990406.csv')
# print(a.loc[a['rank_test_score']==1]['params'].iloc[0])
# print(a.loc[a['rank_test_score']==1]['params'].iloc[1])

# Best models test

In [297]:
# from sklearn.model_selection import GridSearchCV
# from sklearn.pipeline import Pipeline
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.svm import LinearSVC
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import f1_score, classification_report, confusion_matrix

# df_final = df
# # df_final['text_final'] = df_final['text_noPunct'].apply(lambda x : ' '.join(x))
# df_final['text_final'] = df_final['text_neg'].apply(lambda x : ' '.join(x))

# mask_train_test = df_final['sentiment'].notna()

# X_train_valid = df_final.loc[mask_train_test,:]['text_final'].values
# y_train_valid = df_final.loc[mask_train_test,:]['sentiment'].values
# X_test = df_final.loc[~mask_train_test,:]['text_final'].values

# X_train, X_valid, y_train, y_valid = train_test_split(
#     X_train_valid, 
#     y_train_valid, 
#     shuffle=True, 
#     # stratify=y_train_valid, 
#     train_size=0.9, 
#     random_state=42
# )

# vectorizer = TfidfVectorizer(
#     binary = True, 
#     max_df = 1.0, 
#     min_df = 1, 
#     max_features = None, 
#     ngram_range = (1,1), 
#     norm = 'l1', 
#     smooth_idf = True, 
#     stop_words = None, 
#     sublinear_tf = False, 
#     use_idf = True
# )

# model = LinearSVC(
#     C = 1, 
#     class_weight = 'balanced', 
#     dual = False, 
#     fit_intercept = False, 
#     max_iter = 100, 
#     penalty = 'l1', 
#     random_state = 42, 
#     tol = 0.001
# )

# pipe = Pipeline([
#     ('tfidf', vectorizer),
#     ('linearSVC', model)
# ])

# pipe.fit(X_train, y_train)

# f1 = f1_score(y_valid, pipe.predict(X_valid),average='macro')
# report = classification_report(y_valid, pipe.predict(X_valid))
# confusion = confusion_matrix(y_valid, pipe.predict(X_valid))

# print(f1)
# print(report)
# print(confusion)

# MODIFIED: norm and ngram_range

In [302]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report, confusion_matrix

df_final = df
# df_final['text_final'] = df_final['text_noPunct'].apply(lambda x : ' '.join(x))
# df_final['text_final'] = df_final['text_neg'].apply(lambda x : ' '.join(x))
df_final['text_final'] = df_final['text_stemmed_neg'].apply(lambda x : ' '.join(x))
# df_final['text_final'] = df_final['text_noStopwords'].apply(lambda x : ' '.join(x))

mask_train_test = df_final['sentiment'].notna()

X_train_valid = df_final.loc[mask_train_test,:]['text_final'].values
y_train_valid = df_final.loc[mask_train_test,:]['sentiment'].values
X_test = df_final.loc[~mask_train_test,:]['text_final'].values

X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_valid, 
    y_train_valid, 
    shuffle=True, 
    # stratify=y_train_valid, 
    train_size=0.9, 
    random_state=42
)

vectorizer = TfidfVectorizer(
    binary = True, 
    max_df = 1.0, 
    max_features = None, 
    min_df = 1, 
    ngram_range = (1,2), 
    norm = 'l2', 
    smooth_idf = True, 
    stop_words = None, 
    sublinear_tf = False, 
    use_idf = True
)

model = LinearSVC(
    C = 1, 
    class_weight = 'balanced', 
    dual = False, 
    fit_intercept = False, 
    max_iter = 100, 
    penalty = 'l2', 
    random_state = 42, 
    tol = 0.001
)

pipe = Pipeline([
    ('tfidf', vectorizer),
    ('linearSVC', model)
])

pipe.fit(X_train, y_train)

f1 = f1_score(y_valid, pipe.predict(X_valid),average='macro')
report = classification_report(y_valid, pipe.predict(X_valid))
confusion = confusion_matrix(y_valid, pipe.predict(X_valid))

print(f1)
print(report)
print(confusion)

0.7982780155296589
              precision    recall  f1-score   support

         0.0       0.76      0.78      0.77      9395
         1.0       0.84      0.82      0.83     12900

    accuracy                           0.80     22295
   macro avg       0.80      0.80      0.80     22295
weighted avg       0.80      0.80      0.80     22295

[[ 7323  2072]
 [ 2330 10570]]


Best configuration

In [299]:
vectorizer = TfidfVectorizer(
    binary = True, 
    max_df = 1.0, 
    max_features = None, 
    min_df = 1, 
    ngram_range = (1,2), 
    norm = 'l2', 
    smooth_idf = True, 
    stop_words = None, 
    sublinear_tf = False, 
    use_idf = True
)

model = LinearSVC(
    C = 1, 
    class_weight = 'balanced', 
    dual = False, 
    fit_intercept = False, 
    max_iter = 100, 
    penalty = 'l2', 
    random_state = 42, 
    tol = 0.001
)

+ NO stemming: 0.7953890537516292
+ Snowball: 0.7994796670641988
+ Lancaster: 0.7934227929849921
+ Porter: 0.7982780155296589
+ Wordnet: 0.7942085265529459

Results generation

In [303]:
y_pred = pipe.predict(X_test)
pd.DataFrame(y_pred.astype(int),columns=['Predicted']).to_csv("output_79827_porter_noDupl.csv",index_label="Id", header=["Predicted"])

Porter caricato 0.794